# A League of Legends Recommender System

## Part 3: Modeling

In this notebook, we'll build a simple recommender system for the game. The way it works will be the same way it works on various League of Legends analytics sites: given a champion and a role, we count the most popular items and runes chosen. Using Python's `tkinter` library, we'll also provide an extremely simple GUI for the user to interact with. We do not, of course, mean for this to be a competitor to the aformentioned websites, like [op.gg](https://op.gg/) or [u.gg](https://u.gg/), but as a recreation for a personal project. We also simply do not have access to the large amounts of data these websites do.

Originally, we wished to use ML models to create the recommender. One would put in a champion and role, and 6 items would be spat out by the model. However, there were issues with this: we simply did not have enough data to train the model on every single champion in every role. While we have a couple thousand matches, this is simply not nearly enough considering the game has close to 200 champions. What's more, the recommender would output item builds that simply didn't make sense, for instance, recommending a mix of AP and AD items. This is easy enough to fix, as we would label which items were AP/AD and only allow AP/AD champions respectively to choose them. However, this would inherently lock out what may be interesting off-meta builds (like maybe an AD champ building some AP item for the benefits the item gives beyond the AP). There is also no guarantee that even by locking items behind this filter, it would still just come up with completely unviable builds, like Rammus building non-tank AD items.

After deciding that the best players of the game would likely know what to build better than a model would anyway, we opted to go with the current, simpler method of counting.

The recommender can also be used by running `recommender.py`, rather than trudging through this notebook.

---

In [1]:
import os
import numpy as np
import pandas as pd
from utils.func import name_replace
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv(os.path.join('data', 'matches_data.csv'))
df = name_replace(df)
df.head(10)

,Match ID,Game Duration,Game Version,Summoner Name,Summoner Tag,Champion ID,Champion Name,Champion Level,Team,Role,Kills,Deaths,Assists,CS,CS (Jungle),First Blood,First Tower,Objective Stolen,Total Gold Earned,Gold Spent,Gold/Minute,Damage Dealt,% of Team's Damage,Damage Taken,Damage Mitigated,Heal and Shielding,CC Time Dealt,Turret Plates Taken,Turret Takedowns,Vision Score,Rune 1,Rune 2,Rune 3,Rune 4,Sec Rune 1,Sec Rune 2,Stat 1,Stat 2,Stat 3,Summ 1,Summ 2,Item 1,Item 2,Item 3,Item 4,Item 5,Item 6,Triplekills,Quadrakills,Pentakills,Grubs Taken (Team),Heralds Taken (Team),Barons Taken (Team),Dragons Taken (Team),Game Ended in Surrender,Win
0,NA1_5095396461,1404,14.16.612.449,K9 BDE,NA1,122,Darius,14,Blue,TOP,4,2,4,146,0,True,False,0,9688,8608,413.808623,8867,0.135032,14725,11145,0.000000,41,5,2,22,Conqueror,Triumph,Legend: Alacrity,Last Stand,Bone Plating,Unflinching,Attack Speed,Adaptive Force,Health Scaling,Flash,Ghost,Doran's Blade,Trinity Force,0,Sterak's Gage,Mercury's Treads,0,0,0,0,2,1,1,3,True,True
1,NA1_5095396461,1404,14.16.612.449,Hani,zzzzz,141,Kayn,14,Blue,JUNGLE,5,4,9,14,133,False,False,0,10510,9050,448.913791,12101,0.184288,29216,19095,0.000000,362,0,1,14,Conqueror,Triumph,Legend: Haste,Cut Down,Eyeball Collection,Relentless Hunter,Adaptive Force,Adaptive Force,Health Scaling,Flash,Smite,Serylda's Grudge,Eclipse,Serrated Dirk,Mercury's Treads,Long Sword,0,0,0,0,2,1,1,3,True,True
2,NA1_5095396461,1404,14.16.612.449,bean and peanut,rina,893,Aurora,14,Blue,MIDDLE,2,2,13,183,0,False,False,0,9382,7950,400.702712,20446,0.311367,16188,7265,0.000000,214,2,1,19,Electrocute,Taste of Blood,Eyeball Collection,Ultimate Hunter,Manaflow Band,Transcendence,Attack Speed,Adaptive Force,Health Scaling,Flash,Teleport,Doran's Ring,Liandry's Torment,Dark Seal,Malignance,Mercury's Treads,0,0,0,0,2,1,1,3,True,True
3,NA1_5095396461,1404,14.16.612.449,Lucius Artorius,CV1,22,Ashe,14,Blue,BOTTOM,8,1,8,200,4,False,False,0,12423,11200,530.618963,16308,0.248344,7652,3517,0.000000,616,6,4,19,Press the Attack,Absorb Life,Legend: Alacrity,Cut Down,Biscuit Delivery,Approach Velocity,Attack Speed,Adaptive Force,Health,Flash,Cleanse,Doran's Blade,Kraken Slayer,Phantom Dancer,Infinity Edge,Long Sword,Berserker's Greaves,0,0,0,2,1,1,3,True,True
4,NA1_5095396461,1404,14.16.612.449,whiteman enjoyer,2222,147,Seraphine,11,Blue,UTILITY,2,3,13,25,0,False,True,0,7418,6150,316.823169,7943,0.120969,11305,5905,6384.027344,132,5,4,51,Guardian,Font of Life,Bone Plating,Revitalize,Presence of Mind,Cut Down,Attack Speed,Move Speed,Health,Flash,Heal,Echoes of Helia,Ionian Boots of Lucidity,Moonstone Renewer,Celestial Opposition,Faerie Charm,0,0,0,0,2,1,1,3,True,True
5,NA1_5095396461,1404,14.16.612.449,gemi swift,NA1,78,Poppy,13,Red,TOP,5,4,1,146,0,False,False,0,8791,8000,375.489477,11528,0.234211,19294,23368,0.000000,782,1,1,13,Phase Rush,Manaflow Band,Celerity,Scorch,Second Wind,Overgrowth,Adaptive Force,Adaptive Force,Health,Flash,Teleport,Doran's Shield,Mercury's Treads,Bami's Cinder,Iceborn Gauntlet,Abyssal Mask,Cloth Armor,0,0,0,4,0,0,0,True,False
6,NA1_5095396461,1404,14.16.612.449,Giraffe Hugs,NA1,876,Lillia,13,Red,JUNGLE,3,5,5,9,125,False,False,0,8660,8600,369.896944,11602,0.235707,31443,14583,0.000000,437,0,0,27,Conqueror,Triumph,Legend: Haste,Coup de Grace,Cosmic Insight,Magical Footwear,Adaptive Force,Adaptive Force,Health Scaling,Flash,Smite,Riftmaker,Dark Seal,Liandry's Torment,Ionian Boots of Lucidity,Oblivion Orb,0,0,0,0,4,0,0,0,True,False
7,NA1_5095396461,1404,14.16.612.449,Cupic,Senna,99,Lux,13,Red,MIDDLE,1,5,1,188,0,False,False,0,7768,7250,331.782796,12022,0.244247,9643,5812,902.400024,387,1,0,10,Dark Harvest,Cheap Shot,Eyeball Collection,Ultimate Hunter,Cut Down,Presence of Mind,Adaptive Force,Adaptive Force,Health,Flash,Ignite,Luden's Companion,Sorcerer's Shoes,Dark Seal,Doran's Ring,Hextech Alternator,Needlessly Large Rod,0,0,0,4,0,0,0,True,False
8,NA1_5095396461,1404,14.16.612.449,ThëBeesKnees,0001,202,Jhin,12,Red,BOTTOM,2,4,6,163,0,False,Fal

Let's drop all the columns that we don't care about. In this project, we'll only be looking at the most popular runes, items, and summoner spells.

In [3]:
columns_to_keep = [
    'Champion Name', 'Role',
    'Rune 1', 'Rune 2', 'Rune 3', 'Rune 4', 'Sec Rune 1', 'Sec Rune 2', 'Stat 1', 'Stat 2', 'Stat 3',
    'Summ 1', 'Summ 2',
    'Item 1', 'Item 2', 'Item 3', 'Item 4', 'Item 5', 'Item 6'
]

df = df[columns_to_keep]
df['Champion Name'] = df['Champion Name'].str.upper()
df.head(10)

,Champion Name,Role,Rune 1,Rune 2,Rune 3,Rune 4,Sec Rune 1,Sec Rune 2,Stat 1,Stat 2,Stat 3,Summ 1,Summ 2,Item 1,Item 2,Item 3,Item 4,Item 5,Item 6
0,DARIUS,TOP,Conqueror,Triumph,Legend: Alacrity,Last Stand,Bone Plating,Unflinching,Attack Speed,Adaptive Force,Health Scaling,Flash,Ghost,Doran's Blade,Trinity Force,0,Sterak's Gage,Mercury's Treads,0
1,KAYN,JUNGLE,Conqueror,Triumph,Legend: Haste,Cut Down,Eyeball Collection,Relentless Hunter,Adaptive Force,Adaptive Force,Health Scaling,Flash,Smite,Serylda's Grudge,Eclipse,Serrated Dirk,Mercury's Treads,Long Sword,0
2,AURORA,MIDDLE,Electrocute,Taste of Blood,Eyeball Collection,Ultimate Hunter,Manaflow Band,Transcendence,Attack Speed,Adaptive Force,Health Scaling,Flash,Teleport,Doran's Ring,Liandry's Torment,Dark Seal,Malignance,Mercury's Treads,0
3,ASHE,BOTTOM,Press the Attack,Absorb Life,Legend: Alacrity,Cut Down,Biscuit Delivery,Approach Velocity,Attack Speed,Adaptive Force,Health,Flash,Cleanse,Doran's Blade,Kraken Slayer,Phantom Dancer,Infinity Edge,Long Sword,Berserker's Greaves
4,SERAPHINE,UTILITY,Guardian,Font of Life,Bone Plating,Revitalize,Presence of Mind,Cut Down,Attack Speed,Move Speed,Health,Flash,Heal,Echoes of Helia,Ionian Boots of Lucidity,Moonstone Renewer,Celestial Opposition,Faerie Charm,0
5,POPPY,TOP,Phase Rush,Manaflow Band,Celerity,Scorch,Second Wind,Overgrowth,Adaptive Force,Adaptive Force,Health,Flash,Teleport,Doran's Shield,Mercury's Treads,Bami's Cinder,Iceborn Gauntlet,Abyssal Mask,Cloth Armor
6,LILLIA,JUNGLE,Conqueror,Triumph,Legend: Haste,Coup de Grace,Cosmic Insight,Magical Footwear,Adaptive Force,Adaptive Force,Health Scaling,Flash,Smite,Riftmaker,Dark Seal,Liandry's Torment,Ionian Boots of Lucidity,Oblivion Orb,0
7,LUX,MIDDLE,Dark Harvest,Cheap Shot,Eyeball Collection,Ultimate Hunter,Cut Down,Presence of Mind,Adaptive Force,Adaptive Force,Health,Flash,Ignite,Luden's Companion,Sorcerer's Shoes,Dark Seal,Doran's Ring,Hextech Alternator,Needlessly Large Rod
8,JHIN,BOTTOM,Fleet Footwork,Presence of Mind,Legend: Bloodline,Coup de Grace,Celerity,Gathering Storm,Adaptive Force,Adaptive Force,Health Scaling,Barrier,Flash,Doran's Blade,Statikk Shiv,Boots of Swiftness,Cloak of Agility,Pickaxe,B. F. Sword
9,RELL,UTILITY,Aftershock,Demolish,Second Wind,Overgrowth,Cosmic Insight,Hextech Flashtraption,Ability Haste,Move Speed,Health,Ignite,Flash,Boots,Warmog's Armor,Kindlegem,Celestial Opposition,0,0


Because we don't care about component items, like Long Swords, and only want a count of finished items, we'll define all the finished items and drop everything else from the DataFrame.

In [4]:
def filter_items(df, item_columns, finished_items):
    for col in item_columns:
        df[col] = df[col].apply(lambda x: x if x in finished_items else np.nan)
    return df

In [5]:
finished_items = [
    'Berserker\'s Greaves', 'Boots of Swiftness', 'Ionian Boots of Lucidity', 'Mercury\'s Treads', 'Plated Steelcaps', 'Sorcerer\'s Shoes', 'Symbiotic Soles', 'Synchronized Souls', 'Zephyr',
    'Abyssal Mask', 'Archangel\'s Staff', 'Ardent Censer', 'Axiom Arc', 'Banshee\'s Veil', 'Black Cleaver', 'Blackfire Torch', 'Blade of The Ruined King',
    'Bloodsong', 'Bloodthirster', 'Bounty of Worlds', 'Celestial Opposition', 'Chempunk Chainsword', 'Cosmic Drive', 'Cryptbloom', 'Dawncore',
    'Dead Man\'s Plate', 'Death\'s Dance', 'Dream Maker', 'Echoes of Helia', 'Eclipse', 'Edge of Night', 'Essence Reaver', 'Experimental Hexplate',
    'Fimbulwinter', 'Force of Nature', 'Frozen Heart', 'Guardian Angel', 'Guinsoo\'s Rageblade', 'Heartsteel', 'Hextech Rocketbelt', 'Hollow Radiance',
    'Horizon Focus', 'Hubris', 'Hullbreaker', 'Iceborn Gauntlet', 'Immortal Shieldbow', 'Imperial Mandate', 'Infinity Edge', 'Jak\'Sho, The Protean',
    'Kaenic Rookern', 'Knight\'s Vow', 'Kraken Slayer', 'Liandry\'s Torment', 'Lich Bane', 'Locket of the Iron Solari', 'Lord Dominik\'s Regards', 'Luden\'s Companion',
    'Malignance', 'Manamune', 'Maw of Malmortius', 'Mejai\'s Soulstealer', 'Mercurial Scimitar', 'Mikael\'s Blessing', 'Moonstone Renewer', 'Morellonomicon',
    'Mortal Reminder', 'Muramana', 'Nashor\'s Tooth', 'Navori Flickerblade', 'Opportunity', 'Overlord\'s Bloodmail', 'Phantom Dancer', 'Profane Hydra',
    'Rabadon\'s Deathcap', 'Randuin\'s Omen', 'Rapid Firecannon', 'Ravenous Hydra', 'Redemption', 'Riftmaker', 'Rod of Ages', 'Runaan\'s Hurricane',
    'Rylai\'s Crystal Scepter', 'Seraph\'s Embrace', 'Serpent\'s Fang', 'Serylda\'s Grudge', 'Shadowflame', 'Shurelya\'s Battlesong', 'Solstice Sleigh', 'Spear of Shojin',
    'Spirit Visage', 'Staff of Flowing Water', 'Statikk Shiv', 'Sterak\'s Gage', 'Stormsurge', 'Stridebreaker', 'Sundered Sky', 'Sunfire Aegis',
    'Terminus', 'The Collector', 'Thornmail', 'Titanic Hydra', 'Trailblazer', 'Trinity Force', 'Umbral Glaive', 'Unending Despair',
    'Vigilant Wardstone', 'Void Staff', 'Voltaic Cyclosword', 'Warmog\'s Armor', 'Winter\'s Approach', 'Wit\'s End', 'Youmuu\'s Ghostblade', 'Yun Tal Wildarrows',
    'Zaz\'Zak\'s Realmspike', 'Zeke\'s Convergence', 'Zhonya\'s Hourglass'
]
item_columns = ['Item 1', 'Item 2', 'Item 3', 'Item 4', 'Item 5', 'Item 6']
df = filter_items(df, item_columns, finished_items)
df.head(10)

,Champion Name,Role,Rune 1,Rune 2,Rune 3,Rune 4,Sec Rune 1,Sec Rune 2,Stat 1,Stat 2,Stat 3,Summ 1,Summ 2,Item 1,Item 2,Item 3,Item 4,Item 5,Item 6
0,DARIUS,TOP,Conqueror,Triumph,Legend: Alacrity,Last Stand,Bone Plating,Unflinching,Attack Speed,Adaptive Force,Health Scaling,Flash,Ghost,NaN,Trinity Force,NaN,Sterak's Gage,Mercury's Treads,NaN
1,KAYN,JUNGLE,Conqueror,Triumph,Legend: Haste,Cut Down,Eyeball Collection,Relentless Hunter,Adaptive Force,Adaptive Force,Health Scaling,Flash,Smite,Serylda's Grudge,Eclipse,NaN,Mercury's Treads,NaN,NaN
2,AURORA,MIDDLE,Electrocute,Taste of Blood,Eyeball Collection,Ultimate Hunter,Manaflow Band,Transcendence,Attack Speed,Adaptive Force,Health Scaling,Flash,Teleport,NaN,Liandry's Torment,NaN,Malignance,Mercury's Treads,NaN
3,ASHE,BOTTOM,Press the Attack,Absorb Life,Legend: Alacrity,Cut Down,Biscuit Delivery,Approach Velocity,Attack Speed,Adaptive Force,Health,Flash,Cleanse,NaN,Kraken Slayer,Phantom Dancer,Infinity Edge,NaN,Berserker's Greaves
4,SERAPHINE,UTILITY,Guardian,Font of Life,Bone Plating,Revitalize,Presence of Mind,Cut Down,Attack Speed,Move Speed,Health,Flash,Heal,Echoes of Helia,Ionian Boots of Lucidity,Moonstone Renewer,Celestial Opposition,NaN,NaN
5,POPPY,TOP,Phase Rush,Manaflow Band,Celerity,Scorch,Second Wind,Overgrowth,Adaptive Force,Adaptive Force,Health,Flash,Teleport,NaN,Mercury's Treads,NaN,Iceborn Gauntlet,Abyssal Mask,NaN
6,LILLIA,JUNGLE,Conqueror,Triumph,Legend: Haste,Coup de Grace,Cosmic Insight,Magical Footwear,Adaptive Force,Adaptive Force,Health Scaling,Flash,Smite,Riftmaker,NaN,Liandry's Torment,Ionian Boots of Lucidity,NaN,NaN
7,LUX,MIDDLE,Dark Harvest,Cheap Shot,Eyeball Collection,Ultimate Hunter,Cut Down,Presence of Mind,Adaptive Force,Adaptive Force,Health,Flash,Ignite,Luden's Companion,Sorcerer's Shoes,NaN,NaN,NaN,NaN
8,JHIN,BOTTOM,Fleet Footwork,Presence of Mind,Legend: Bloodline,Coup de Grace,Celerity,Gathering Storm,Adaptive Force,Adaptive Force,Health Scaling,Barrier,Flash,NaN,Statikk Shiv,Boots of Swiftness,NaN,NaN,NaN
9,RELL,UTILITY,Aftershock,Demolish,Second Wind,Overgrowth,Cosmic Insight,Hextech Flashtraption,Ability Haste,Move Speed,Health,Ignite,Flash,NaN,Warmog's Armor,NaN,Celestial Opposition,NaN,NaN


Next, we'll need a dictionary to sort out all the runes into their proper sections, because just a count of every possible rune is very helpful. There are 5 rune pages in the game, and each one provides different runes, so we'll need to choose the most popular rune pages and a count of the runes within that rune page.

In [6]:
rune_section_map = {
    'Precision': {
        'Keystone': ['Press the Attack', 'Fleet Footwork', 'Conqueror'],
        'Slot 1': ['Absorb Life', 'Triumph', 'Presence of Mind'],
        'Slot 2': ['Legend: Alacrity', 'Legend: Haste', 'Legend: Bloodline'],
        'Slot 3': ['Coup de Grace', 'Cut Down', 'Last Stand']
    },
    'Domination': {
        'Keystone': ['Electrocute', 'Dark Harvest', 'Hail of Blades'],
        'Slot 1': ['Cheap Shot', 'Taste of Blood', 'Sudden Impact'],
        'Slot 2': ['Zombie Ward', 'Ghost Poro', 'Eyeball Collection'],
        'Slot 3': ['Treasure Hunter', 'Relentless Hunter', 'Ultimate Hunter']
    },
    'Sorcery': {
        'Keystone': ['Summon Aery', 'Arcane Comet', 'Phase Rush'],
        'Slot 1': ['Nullifying Orb', 'Manaflow Band', 'Nimbus Cloak'],
        'Slot 2': ['Transcendence', 'Celerity', 'Absolute Focus'],
        'Slot 3': ['Scorch', 'Waterwalking', 'Gathering Storm']
    },
    'Resolve': {
        'Keystone': ['Grasp of the Undying', 'Aftershock', 'Guardian'],
        'Slot 1': ['Demolish', 'Font of Life', 'Shield Bash'],
        'Slot 2': ['Conditioning', 'Second Wind', 'Bone Plating'],
        'Slot 3': ['Overgrowth', 'Revitalize', 'Unflinching']
    },
    'Inspiration': {
        'Keystone': ['Glacial Augment', 'Unsealed Spellbook', 'First Strike'],
        'Slot 1': ['Hextech Flashtraption', 'Magical Footwear', 'Cash Back'],
        'Slot 2': ['Triple Tonic', 'Time Warp Tonic', 'Biscuit Delivery'],
        'Slot 3': ['Cosmic Insight', 'Approach Velocity', 'Jack of All Trades']
    }
}

sec_rune_section_map = {
    section: {slot: runes for slot, runes in slots.items() if slot != 'Keystone'}
    for section, slots in rune_section_map.items()
}

In [7]:
def aggregate_runes(df, primary_rune_columns, secondary_rune_columns, rune_section_map, sec_rune_section_map):
    # Initialize dictionaries to store rune counts
    rune_sections = {section: {slot: {rune: 0 for rune in runes} for slot, runes in slots.items()}
                     for section, slots in rune_section_map.items()}
    sec_rune_sections = {section: {slot: {rune: 0 for rune in runes} for slot, runes in slots.items()}
                         for section, slots in sec_rune_section_map.items()}

    # Count primary runes
    for column, slot in zip(primary_rune_columns, ['Keystone', 'Slot 1', 'Slot 2', 'Slot 3']):
        rune_counts = df[column].value_counts()

        for rune, count in rune_counts.items():
            for section, slots in rune_section_map.items():
                if rune in rune_sections[section][slot]:
                    rune_sections[section][slot][rune] += count
    
    # Combine counts from Sec Rune 1 and Sec Rune 2 for secondary runes
    combined_sec_runes = pd.concat([df['Sec Rune 1'], df['Sec Rune 2']])
    combined_sec_rune_counts = combined_sec_runes.value_counts()

    # Count secondary runes (excluding keystones) using combined counts
    for rune, count in combined_sec_rune_counts.items():
        for section, slots in sec_rune_section_map.items():
            for slot, runes in slots.items():
                if rune in runes:
                    sec_rune_sections[section][slot][rune] += count
    
    return rune_sections, sec_rune_sections

We'll do the same thing for the stats.

In [8]:
stat_section_map = {
    'Offense': ['Adaptive Force', 'Attack Speed', 'Ability Haste'],
    'Flex': ['Adaptive Force', 'Move Speed', 'Health Scaling'],
    'Defense': ['Health', 'Tenacity and Slow Resist', 'Health Scaling']
}

In [9]:
def aggregate_stats(df, stat_section_map):
    stat_sections = {category: {stat: 0 for stat in stats} for category, stats in stat_section_map.items()}

    for stat_column, category in zip(['Stat 1', 'Stat 2', 'Stat 3'], ['Offense', 'Flex', 'Defense']):
        stat_counts = df[stat_column].value_counts()

        for stat, count in stat_counts.items():
            if stat in stat_sections[category]:
                stat_sections[category][stat] += count
    
    return stat_sections

Now, we define the function for counting objects generally (for the items and summoner spells).

In [10]:
def aggregate_frequencies(df, column_list):
    combined_series = pd.concat([df[col] for col in column_list], ignore_index=True)
    frequencies = combined_series.dropna().value_counts()       # Drop NaN values before counting
    return frequencies

Finally, we put it all together into our system. It should display the top 10 most popular items (6 is not enough because there might be items that are mutually exclusive within the list, like two different types of boots), the most popular primary rune page and the rune counts within that page, the most popular secondary rune page and the rune counts within that page, the most popular stats, and the most popular summoner spells.

In [11]:
def recommend_based_on_champion(df, champion_name, role, rune_section_map, sec_rune_section_map, stat_section_map):
    # Filter data by role and champion
    df_filtered = df[(df['Champion Name'] == champion_name) & (df['Role'] == role)]
    
    if df_filtered.empty:
        return f'No data available for champion {champion_name} in role {role}.'
    
    item_columns = ['Item 1', 'Item 2', 'Item 3', 'Item 4', 'Item 5', 'Item 6']
    rune_columns = ['Rune 1', 'Rune 2', 'Rune 3', 'Rune 4']
    sec_rune_columns = ['Sec Rune 1', 'Sec Rune 2']
    stat_columns = ['Stat 1', 'Stat 2', 'Stat 3']
    summoner_spell_columns = ['Summ 1', 'Summ 2']
    
    # Aggregating different data types
    item_frequencies = aggregate_frequencies(df_filtered, item_columns)
    rune_frequencies, sec_rune_frequencies = aggregate_runes(df_filtered, rune_columns, sec_rune_columns, rune_section_map, sec_rune_section_map)
    stat_frequencies = aggregate_stats(df_filtered, stat_section_map)
    summoner_spell_frequencies = aggregate_frequencies(df_filtered, summoner_spell_columns)
    
    return {
        'Items': item_frequencies,
        'Runes': rune_frequencies,
        'Secondary Runes': sec_rune_frequencies,
        'Stats': stat_frequencies,
        'Summoner Spells': summoner_spell_frequencies
    }

In [12]:
def display_most_popular(recommendations, num_items=10):

    if type(recommendations) == str:
        return recommendations

    def get_most_popular_runes(rune_sections):
        most_popular = {}
        max_count = 0
        for section, slots in rune_sections.items():
            section_count = sum(max(runes.values()) for slot, runes in slots.items())
            if section_count > max_count:
                max_count = section_count
                most_popular = {section: slots}
        return most_popular

    res = ''

    res += 'Most Popular Items:'
    res += f'\n{'-' * 20}'
    res += f'\n{recommendations['Items'].head(num_items).to_string()}'

    res += '\n\n\nMost Popular Runes:'
    res += f'\n{'-' * 20}'
    most_popular_runes = get_most_popular_runes(recommendations.get('Runes', {}))
    for section, slots in most_popular_runes.items():
        res += f'\n{section}:'
        for slot, runes in slots.items():
            res += f'\n  {slot}:'
            for rune, count in runes.items():
                res += f'\n    {rune}: {count}'

    res += '\n\n\nMost Popular Secondary Runes:'
    res += f'\n{'-' * 20}'
    most_popular_sec_runes = get_most_popular_runes(recommendations.get('Secondary Runes', {}))
    for section, slots in most_popular_sec_runes.items():
        res += f'\n{section}:'
        for slot, runes in slots.items():
            res += f'\n  {slot}:'
            for rune, count in runes.items():
                res += f'\n    {rune}: {count}'

    res += '\n\n\nMost Popular Stats:'
    res += f'\n{'-' * 20}'
    stat_sections = recommendations.get('Stats', {})
    for category, stats in stat_sections.items():
        res += f'\n{category}:'
        for stat, count in stats.items():
            res += f'\n  {stat}: {count}'

    res += '\n\n\nMost Popular Summoner Spells:'
    res += f'\n{'-' * 20}'
    res += f'\n{recommendations['Summoner Spells'].to_string()}'

    return res

In [13]:
champion_name = 'AHRI'
role = 'MIDDLE'
recommendations = recommend_based_on_champion(df, champion_name, role, rune_section_map, sec_rune_section_map, stat_section_map)
display = display_most_popular(recommendations)
print(display)

Most Popular Items:
--------------------
Malignance                  197
Sorcerer's Shoes            103
Ionian Boots of Lucidity    100
Horizon Focus                59
Lich Bane                    56
Zhonya's Hourglass           43
Liandry's Torment            41
Rabadon's Deathcap           41
Shadowflame                  28
Cryptbloom                   23


Most Popular Runes:
--------------------
Domination:
  Keystone:
    Electrocute: 194
    Dark Harvest: 1
    Hail of Blades: 0
  Slot 1:
    Cheap Shot: 3
    Taste of Blood: 179
    Sudden Impact: 13
  Slot 2:
    Zombie Ward: 8
    Ghost Poro: 0
    Eyeball Collection: 187
  Slot 3:
    Treasure Hunter: 5
    Relentless Hunter: 1
    Ultimate Hunter: 189


Most Popular Secondary Runes:
--------------------
Sorcery:
  Slot 1:
    Nullifying Orb: 1
    Manaflow Band: 97
    Nimbus Cloak: 0
  Slot 2:
    Transcendence: 112
    Celerity: 1
    Absolute Focus: 4
  Slot 3:
    Scorch: 17
    Waterwalking: 1
    Gathering Storm: 21



It seems nice and fulfills its purpose. All that's left to do now is create a front-end GUI for easier use. We define the function here.

In [14]:
import tkinter as tk
from tkinter import ttk

def create_recommender(df, champion_names, roles, rune_section_map, sec_rune_section_map, stat_section_map, num_items=10):
    def on_button_click(event=None):
        champion = champion_combobox.get().upper()
        role = role_combobox.get().upper()
        recommendations = recommend_based_on_champion(df, champion, role, rune_section_map, sec_rune_section_map, stat_section_map)
        result = display_most_popular(recommendations, num_items)
        text_window.delete(1.0, tk.END)     # Clear previous text
        text_window.insert(tk.END, result)

    # Create the main window
    root = tk.Tk()
    root.title('Build Recommendation')

    # Create and place the Champion Name dropdown
    champion_label = tk.Label(root, text='Champion Name:')
    champion_label.grid(row=0, column=0, padx=10, pady=10, sticky='e')

    champion_combobox = ttk.Combobox(root, values=champion_names)
    champion_combobox.grid(row=0, column=1, padx=10, pady=10, sticky='w')

    # Create and place the Role dropdown
    role_label = tk.Label(root, text='Role:')
    role_label.grid(row=1, column=0, padx=10, pady=10, sticky='e')

    role_combobox = ttk.Combobox(root, values=roles)
    role_combobox.grid(row=1, column=1, padx=10, pady=10, sticky='w')
    role_combobox.set(roles[0])

    # Create and place the button
    recommendations_button = tk.Button(root, text='Get Recommendations', command=on_button_click)
    recommendations_button.grid(row=2, column=0, columnspan=2, padx=10, pady=10)

    # Bind Enter key to the button click event
    root.bind('<Return>', on_button_click)

    # Escape to close application
    root.bind('<Escape>', lambda event: root.destroy())

    # Create a frame to hold the text window and scrollbar
    text_frame = tk.Frame(root)
    text_frame.grid(row=3, column=0, columnspan=2, padx=10, pady=10)

    # Create and place the text window
    text_window = tk.Text(text_frame, wrap=tk.WORD, height=50, width=50)
    text_window.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

    # Create and place the scrollbar
    scrollbar = tk.Scrollbar(text_frame, command=text_window.yview)
    scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

    # Configure the text window to use the scrollbar
    text_window.config(yscrollcommand=scrollbar.set)

    # Focus the main window
    root.after(50, root.focus_force())
    root.after(50, lambda: champion_combobox.focus_set())

    # Run the application
    root.mainloop()

Lastly, the GUI can be accessed by running the following code block.

In [15]:
champion_names = sorted([champion for champion in df['Champion Name'].unique()])
roles = [role for role in df['Role'].unique()]

create_recommender(df, champion_names, roles, rune_section_map, sec_rune_section_map, stat_section_map)